In [3]:
# This connects the python files to the main.js file
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import json

# Read the JavaScript data from the file
with open('main.js', 'r') as file:
    js_code = file.read()


# Extract the JavaScript array parts
start_index1 = js_code.find('[')
end_index1 = js_code.find(']', start_index1)
js_array_str1 = js_code[start_index1:end_index1 + 1]

start_index2 = js_code.find('[', end_index1)
end_index2 = js_code.find(']', start_index2)
js_array_str2 = js_code[start_index2:end_index2 + 1]

# Convert the JavaScript array strings to Python lists
numbers1 = json.loads(js_array_str1)
numbers2 = json.loads(js_array_str2)

print("Red Team:", numbers1)  # Output: [1, 2, 3, 4, 5]
print("Blue Team:", numbers2)  # Output: [6, 7, 8, 9, 10]


Red Team: ['Phoenix', 'Clove', 'Reyna', 'Skye', 'Cypher']
Blue Team: ['Clove', 'Sova', 'Reyna', 'Neon', 'Cypher']


In [71]:
import requests

# Make a GET request to the API endpoint
response = requests.get('https://api.henrikdev.xyz/valorant/v3/matches/na/furbsa/cam')
data = ""

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Convert the response to JSON format
    data = response.json()
else:
    print('Failed to retrieve data from the API:', response.status_code)

# Initialize dictionaries to store the required information
map_win_rate = {}
agent_win_rate = {}
map_kill_death = {}
agent_kill_death = {}

# Loop through each match
for match in data['data']:
    # Extract map name and winning team
    map_name = match['metadata']['map']
    winning_team = 'Red' if match['teams']['red']['has_won'] else 'Blue'

    # Update map win rate
    if map_name not in map_win_rate:
        map_win_rate[map_name] = {'total_matches': 0, 'wins': 0}
    map_win_rate[map_name]['total_matches'] += 1
    if match['teams']['red']['has_won'] or match['teams']['blue']['has_won']:
        map_win_rate[map_name]['wins'] += 1

    # Loop through each player
    for player_data in match['players']['all_players']:
        agent = player_data['character']
        team = player_data['team']
        kills = player_data['stats']['kills']
        deaths = player_data['stats']['deaths']

        # Update agent win rate
        if agent not in agent_win_rate:
            agent_win_rate[agent] = {'total_matches': 0, 'wins': 0}
        agent_win_rate[agent]['total_matches'] += 1
        if team == winning_team:
            agent_win_rate[agent]['wins'] += 1

        # Update map kill/death
        if map_name not in map_kill_death:
            map_kill_death[map_name] = {'total_kills': 0, 'total_deaths': 0}
        map_kill_death[map_name]['total_kills'] += kills
        map_kill_death[map_name]['total_deaths'] += deaths

        # Update agent kill/death
        if agent not in agent_kill_death:
            agent_kill_death[agent] = {'total_kills': 0, 'total_deaths': 0}
        agent_kill_death[agent]['total_kills'] += kills
        agent_kill_death[agent]['total_deaths'] += deaths

# Calculate win rates and kill/death ratios
for map_name, win_data in map_win_rate.items():
    total_matches = win_data['total_matches']
    wins = win_data['wins']
    win_rate = wins / total_matches if total_matches > 0 else 0
    map_win_rate[map_name]['win_rate'] = win_rate

for agent, win_data in agent_win_rate.items():
    total_matches = win_data['total_matches']
    wins = win_data['wins']
    win_rate = wins / total_matches if total_matches > 0 else 0
    agent_win_rate[agent]['win_rate'] = win_rate

for map_name, kd_data in map_kill_death.items():
    total_kills = kd_data['total_kills']
    total_deaths = kd_data['total_deaths']
    kill_death_ratio = total_kills / total_deaths if total_deaths > 0 else 0
    map_kill_death[map_name]['kill_death_ratio'] = kill_death_ratio

for agent, kd_data in agent_kill_death.items():
    total_kills = kd_data['total_kills']
    total_deaths = kd_data['total_deaths']
    kill_death_ratio = total_kills / total_deaths if total_deaths > 0 else 0
    agent_kill_death[agent]['kill_death_ratio'] = kill_death_ratio

 # Create DataFrames for each category
map_win_rate_df = pd.DataFrame.from_dict(map_win_rate, orient='index', columns=['win_rate'])
agent_win_rate_df = pd.DataFrame.from_dict(agent_win_rate, orient='index', columns=['win_rate'])
map_kill_death_df = pd.DataFrame.from_dict(map_kill_death, orient='index', columns=['kill_death_ratio'])
agent_kill_death_df = pd.DataFrame.from_dict(agent_kill_death, orient='index', columns=['kill_death_ratio'])

# Now you have dictionaries containing the required information
# map_win_rate: Win rate of each map
# agent_win_rate: Win rate of each agent
# map_kill_death: Kill/death ratio on each map
# agent_kill_death: Kill/death ratio of each agent

In [72]:
print("Win rate of each map:")
for map_name, win_data in map_win_rate.items():
    print(f"{map_name}: {win_data['win_rate']:.2%}")

# Print the win rate of each agent
print("\nWin rate of each agent:")
for agent, win_data in agent_win_rate.items():
    print(f"{agent}: {win_data['win_rate']:.2%}")

# Print the kill/death ratio on each map
print("\nKill/death ratio on each map:")
for map_name, kd_data in map_kill_death.items():
    print(f"{map_name}: {kd_data['kill_death_ratio']:.2f}")

# Print the kill/death ratio of each agent
print("\nKill/death ratio of each agent:")
for agent, kd_data in agent_kill_death.items():
    print(f"{agent}: {kd_data['kill_death_ratio']:.2f}")

Win rate of each map:
Breeze: 100.00%
Bind: 100.00%
Lotus: 0.00%
Sunset: 100.00%

Win rate of each agent:
Reyna: 60.00%
Viper: 25.00%
Cypher: 50.00%
Sova: 33.33%
Jett: 40.00%
Fade: 0.00%
Skye: 100.00%
Raze: 25.00%
Clove: 50.00%
Astra: 0.00%
Gekko: 25.00%
Breach: 0.00%
Killjoy: 0.00%
KAY/O: 0.00%
Deadlock: 0.00%
Chamber: 0.00%
Phoenix: 100.00%
Omen: 0.00%
Yoru: 0.00%

Kill/death ratio on each map:
Breeze: 1.00
Bind: 0.99
Lotus: 1.00
Sunset: 1.00

Kill/death ratio of each agent:
Reyna: 1.19
Viper: 1.01
Cypher: 1.19
Sova: 0.88
Jett: 0.88
Fade: 0.47
Skye: 1.12
Raze: 0.81
Clove: 0.86
Astra: 0.87
Gekko: 0.98
Breach: 1.05
Killjoy: 0.67
KAY/O: 1.00
Deadlock: 0.67
Chamber: 2.35
Phoenix: 1.68
Omen: 0.39
Yoru: 0.47


In [79]:
import requests

# Make a GET request to the API endpoint
response = requests.get('https://api.henrikdev.xyz/valorant/v3/matches/na/ImSuperSped/7033')
data = ""

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Convert the response to JSON format
    data = response.json()
else:
    print('Failed to retrieve data from the API:', response.status_code)

# Initialize dictionaries to store the required information
map_win_rate = {}
agent_win_rate = {}
map_kill_death = {}
agent_kill_death = {}

# Loop through each match
for match in data['data']:
    # Extract map name and winning team
    map_name = match['metadata']['map']
    winning_team = 'Red' if match['teams']['red']['has_won'] else 'Blue'

    # Update map win rate
    if map_name not in map_win_rate:
        map_win_rate[map_name] = {'total_matches': 0, 'wins': 0}
    map_win_rate[map_name]['total_matches'] += 1
    if match['teams']['red']['has_won'] or match['teams']['blue']['has_won']:
        map_win_rate[map_name]['wins'] += 1

    # Loop through each player
    for player_data in match['players']['all_players']:
        agent = player_data['character']
        team = player_data['team']
        kills = player_data['stats']['kills']
        deaths = player_data['stats']['deaths']

        # Update agent win rate
        if agent not in agent_win_rate:
            agent_win_rate[agent] = {'total_matches': 0, 'wins': 0}
        agent_win_rate[agent]['total_matches'] += 1
        if team == winning_team:
            agent_win_rate[agent]['wins'] += 1

        # Update map kill/death
        if map_name not in map_kill_death:
            map_kill_death[map_name] = {'total_kills': 0, 'total_deaths': 0}
        map_kill_death[map_name]['total_kills'] += kills
        map_kill_death[map_name]['total_deaths'] += deaths

        # Update agent kill/death
        if agent not in agent_kill_death:
            agent_kill_death[agent] = {'total_kills': 0, 'total_deaths': 0}
        agent_kill_death[agent]['total_kills'] += kills
        agent_kill_death[agent]['total_deaths'] += deaths

# Calculate win rates and kill/death ratios
for map_name, win_data in map_win_rate.items():
    total_matches = win_data['total_matches']
    wins = win_data['wins']
    win_rate = wins / total_matches if total_matches > 0 else 0
    map_win_rate[map_name]['win_rate'] = win_rate

for agent, win_data in agent_win_rate.items():
    total_matches = win_data['total_matches']
    wins = win_data['wins']
    win_rate = wins / total_matches if total_matches > 0 else 0
    agent_win_rate[agent]['win_rate'] = win_rate

for map_name, kd_data in map_kill_death.items():
    total_kills = kd_data['total_kills']
    total_deaths = kd_data['total_deaths']
    kill_death_ratio = total_kills / total_deaths if total_deaths > 0 else 0
    map_kill_death[map_name]['kill_death_ratio'] = kill_death_ratio

for agent, kd_data in agent_kill_death.items():
    total_kills = kd_data['total_kills']
    total_deaths = kd_data['total_deaths']
    kill_death_ratio = total_kills / total_deaths if total_deaths > 0 else 0
    agent_kill_death[agent]['kill_death_ratio'] = kill_death_ratio

In [80]:
print("Win rate of each map:")
for map_name, win_data in map_win_rate.items():
    print(f"{map_name}: {win_data['win_rate']:.2%}")

# Print the win rate of each agent
print("\nWin rate of each agent:")
for agent, win_data in agent_win_rate.items():
    print(f"{agent}: {win_data['win_rate']:.2%}")

# Print the kill/death ratio on each map
print("\nKill/death ratio on each map:")
for map_name, kd_data in map_kill_death.items():
    print(f"{map_name}: {kd_data['kill_death_ratio']:.2f}")

# Print the kill/death ratio of each agent
print("\nKill/death ratio of each agent:")
for agent, kd_data in agent_kill_death.items():
    print(f"{agent}: {kd_data['kill_death_ratio']:.2f}")

map_win_rate_df2 = pd.DataFrame.from_dict(map_win_rate, orient='index', columns=['win_rate'])
agent_win_rate_df2 = pd.DataFrame.from_dict(agent_win_rate, orient='index', columns=['win_rate'])
map_kill_death_df2 = pd.DataFrame.from_dict(map_kill_death, orient='index', columns=['kill_death_ratio'])
agent_kill_death_df2 = pd.DataFrame.from_dict(agent_kill_death, orient='index', columns=['kill_death_ratio'])

Win rate of each map:
Icebox: 100.00%
Lotus: 100.00%
Sunset: 0.00%
Breeze: 0.00%

Win rate of each agent:
Sova: 0.00%
Phoenix: 0.00%
Jett: 60.00%
Reyna: 40.00%
Clove: 60.00%
Killjoy: 20.00%
Viper: 25.00%
Gekko: 75.00%
Harbor: 0.00%
Breach: 0.00%
Raze: 0.00%
Chamber: 50.00%
Skye: 33.33%
Brimstone: 0.00%
Neon: 0.00%
Yoru: 0.00%
Iso: 0.00%
Sage: 0.00%
Fade: 0.00%
Omen: 0.00%

Kill/death ratio on each map:
Icebox: 1.00
Lotus: 1.00
Sunset: 1.00
Breeze: 1.00

Kill/death ratio of each agent:
Sova: 1.29
Phoenix: 1.28
Jett: 1.31
Reyna: 1.01
Clove: 1.06
Killjoy: 0.80
Viper: 1.21
Gekko: 1.03
Harbor: 1.26
Breach: 0.58
Raze: 0.55
Chamber: 1.19
Skye: 0.77
Brimstone: 1.07
Neon: 1.44
Yoru: 0.89
Iso: 1.00
Sage: 0.71
Fade: 1.04
Omen: 0.26


In [81]:
# Print the DataFrames
print("DataFrame for win rate of each map:")
print(map_win_rate_df)

print("\nDataFrame for win rate of each agent:")
print(agent_win_rate_df)

print("\nDataFrame for kill/death ratio on each map:")
print(map_kill_death_df)

print("\nDataFrame for kill/death ratio of each agent:")
print(agent_kill_death_df)

DataFrame for win rate of each map:
        win_rate
Bind         1.0
Breeze       1.0
Lotus        0.0
Sunset       1.0

DataFrame for win rate of each agent:
          win_rate
Astra     0.000000
Breach    0.000000
Chamber   0.000000
Clove     0.500000
Cypher    0.500000
Deadlock  0.000000
Fade      0.000000
Gekko     0.250000
Jett      0.400000
KAY/O     0.000000
Killjoy   0.000000
Omen      0.000000
Phoenix   1.000000
Raze      0.250000
Reyna     0.600000
Skye      1.000000
Sova      0.333333
Viper     0.250000
Yoru      0.000000

DataFrame for kill/death ratio on each map:
        kill_death_ratio
Bind            0.992933
Breeze          1.000000
Lotus           1.000000
Sunset          1.000000

DataFrame for kill/death ratio of each agent:
          kill_death_ratio
Astra             0.866667
Breach            1.045455
Chamber           2.352941
Clove             0.855422
Cypher            1.190000
Deadlock          0.666667
Fade              0.466667
Gekko             0.984127


In [82]:
# Assign weights to win rate and kill/death ratio
win_rate_weight = 0.5
kd_ratio_weight = 0.5

# Create a DataFrame to store the combined scores for each agent on each map
combined_scores_df = pd.DataFrame(index=agent_win_rate_df.index, columns=map_win_rate_df.index)

# Calculate combined scores for each agent on each map
for map_name in map_win_rate_df.index:
    win_rate_map = map_win_rate_df.loc[map_name, 'win_rate']
    kd_ratio_map = map_kill_death_df.loc[map_name, 'kill_death_ratio']
    
    for agent in agent_win_rate_df.index:
        win_rate_agent = agent_win_rate_df.loc[agent, 'win_rate']
        kd_ratio_agent = agent_kill_death_df.loc[agent, 'kill_death_ratio']
        
        # Calculate combined score using weighted sum
        combined_score = (win_rate_weight * win_rate_agent + kd_ratio_weight * kd_ratio_agent) / 2
        combined_scores_df.loc[agent, map_name] = combined_score

# Find the best agent for each map based on the combined score
best_agents_for_each_map = combined_scores_df.idxmax(axis=0)

# Print the list of agents in order of the best agent to pick for every map
print("Best Agents for Each Map:")
for map_name, best_agent in best_agents_for_each_map.items():
    print(f"{map_name}: {best_agent}")

Best Agents for Each Map:
Bind: Phoenix
Breeze: Phoenix
Lotus: Phoenix
Sunset: Phoenix


In [83]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Concatenate the DataFrames
combined_map_win_rate_df = pd.concat([map_win_rate_df, map_win_rate_df2], axis=1)
combined_agent_win_rate_df = pd.concat([agent_win_rate_df, agent_win_rate_df2], axis=1)
combined_map_kill_death_df = pd.concat([map_kill_death_df, map_kill_death_df2], axis=1)
combined_agent_kill_death_df = pd.concat([agent_kill_death_df, agent_kill_death_df2], axis=1)

# Concatenate all features
X = pd.concat([combined_map_win_rate_df, combined_map_kill_death_df,
               combined_agent_win_rate_df, combined_agent_kill_death_df], axis=1)
# Target variable is the best agent for each map
y = X.idxmax(axis=1)

# Handle missing values
imputer = SimpleImputer(strategy='mean')  # You can choose another strategy if needed
X_imputed = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Initialize individual classifiers
classifier1 = RandomForestClassifier()
classifier2 = RandomForestClassifier()

# Create the ensemble model using a Voting Classifier
ensemble_model = VotingClassifier(estimators=[
    ('rf1', classifier1), ('rf2', classifier2)], voting='hard')

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = ensemble_model.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0
